# Project 

In [111]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

**Part 1.**

Loading the data :

In [112]:
food_inspections = pd.read_csv('data/food-inspections.csv', sep=',')

**Part 2.**

Drop column we don't need :

In [113]:
food_inspections = food_inspections.drop(['Zip', 'Zip Codes','Historical Wards 2003-2015', 'Community Areas', 'Census Tracts','Wards','AKA Name','State','City','Address'], axis=1)

In [114]:
food_inspections.head()

,Inspection ID,DBA Name,License #,Facility Type,Risk,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2320977,TODD APPEL BAR AND COCKTAIL,2589615.0,Grocery Store,Risk 2 (Medium),2019-11-01T00:00:00.000,Canvass,No Entry,NaN,41.988060,-87.703704,"{'latitude': '-87.70370445181302', 'longitude'..."
1,2320927,Sayre Language Acad.,29271.0,School,Risk 1 (High),2019-11-01T00:00:00.000,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.914124,-87.798757,"{'latitude': '-87.79875666766955', 'longitude'..."
2,2320911,Hitch Elementary School,23811.0,School,Risk 1 (High),2019-11-01T00:00:00.000,Canvass Re-Inspection,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.983678,-87.779086,"{'latitude': '-87.77908614427155', 'longitude'..."
3,2320918,BEEFSTEAK,2698445.0,Restaurant,Risk 1 (High),2019-11-01T00:00:00.000,License,Pass,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,41.895692,-87.620143,"{'latitude': '-87.62014286048188', 'longitude'..."
4,2320916,ST. CONSTANCE ELEMENTARY SCHOOL,1878492.0,School,Risk 1 (High),2019-11-01T00:00:00.000,Canvass,No Entry,NaN,41.970176,-87.775099,"{'latitude': '-87.77509944904432', 'longitude'..."


In [115]:
food_inspections['Inspection ID'].isnull().values.any()

False

In [116]:
food_inspections['Inspection ID'].count()

195212

**Part 3.**

Analyse facilities :

Remarque :
Selon Le food Inspection, seul ces facilites devrais être enregistrée.

    1.bakery, 
    2.banquet hall
    3.candy store
    4.caterer
    5.coffee shop
    6.day care center (for ages less than 2)
    7.day care center (for ages 2 – 6)
    8.day care center (combo, for ages less than 2 and 2 – 6 combined)
    9.gas station
    10.Golden Diner
    11.grocery store
    12.hospital
    13.long term care center(nursing home)
    14.liquor store
    15.mobile food dispenser
    16.restaurant
    17.paleteria
    18.school
    19.shelter
    20.tavern
    21.social club
    22.wholesaler
    23.Wrigley Field Rooftop. 



In [117]:
Unique_license = food_inspections.drop_duplicates(subset="License #")
Unique_license.groupby('Facility Type')['Facility Type'].count().sort_values(ascending=False).head(50)

Facility Type
Restaurant                                        19822
Grocery Store                                      6167
School                                             1166
Children's Services Facility                        636
Daycare (2 - 6 Years)                               584
Daycare Above and Under 2 Years                     496
Bakery                                              488
Mobile Food Dispenser                               468
Liquor                                              407
Wholesale                                           260
Long Term Care                                      257
Catering                                            218
Daycare Combo 1586                                  182
Shared Kitchen User (Long Term)                     176
Mobile Food Preparer                                160
TAVERN                                              123
Hospital                                             98
Golden Diner                      

On ne vas s'interesser que au Facility type : Restaurant,Grocery Store, School et hopital

In [118]:
Facility_list =  ['Restaurant', 'Grocery Store', 'School', 'Hospital']
Unique_license = Unique_license[Unique_license['Facility Type'].isin(Facility_list)]
Unique_license = Unique_license[Unique_license.duplicated(subset='Location')]

In [121]:
Unique_license.groupby('Facility Type')['Facility Type'].count().sort_values(ascending=False).head()

Facility Type
Restaurant       10628
Grocery Store     2820
School             202
Hospital            64
Name: Facility Type, dtype: int64

Il s'agit d'un dataset ne comprenant que les Unique commerce.

In [123]:
Facility_Type_Db = Unique_license[['Facility Type','Latitude','Longitude']]

In [125]:
#Drop la ou y a pas de latitude et longitude
Facility_Type_Db.dropna(subset=['Latitude'], inplace=True)
Facility_Type_Db.head()

,Facility Type,Latitude,Longitude
11,Restaurant,41.895692,-87.620143
28,Restaurant,42.008536,-87.914428
32,Restaurant,42.008536,-87.914428
38,Restaurant,41.966148,-87.688683
41,Restaurant,41.931918,-87.725545


**Facility_type_Db:** Est un dataset Avec uniquement les Facility Unique

**Part 3** : Inspection

1. Moyenne Inspection Pour chaque Facility type
2. Moyenne Inspection pour chaque Facility type au cours des année
3. Moyenne Inspection des 5 plus grandes institutions


0) Drop Nan dans localisation
   
1) Moyenne Inspection pour chaque facility type

Facility_filtered dataset = {restaurant, Grocery Store, School, Hospital}
Nbrs Inspection / Nbrs Facility  = Moyenne Inspection pour Facility

2) Au cours des année

To_date

3) Moyenne inspection des 5 plus grandes institutions

Institution_filtered dataset = {1...,2...,3...,4...,5...}
-> Unifier les noms